First, we load the configuration parameter and initialize a molecular graph generator.

In [1]:
import json
from dig.ggraph.method import GraphDF
with open('config/prop_opt_plogp_config_dict.json') as f:
    conf = json.load(f)
runner = GraphDF()

ModuleNotFoundError: No module named 'dig'

Next, we load the pre-trained model from 'saved_ckpts/prop_opt/pretrain_plogp.pth', and start fine-tuning.

In [ ]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
pretrain_path = 'saved_ckpts/prop_opt/pretrain_plogp.pth'
lr = 0.0001
wd = 0
warm_up = 0
max_iters = 200
save_interval = 20
save_dir = 'prop_opt_plogp'
runner.train_prop_opt(lr=lr, wd=wd, max_iters=max_iters, warm_up=warm_up,
    model_conf_dict=model_conf_dict, pretrain_path=pretrain_path,
    save_interval=save_interval, save_dir=save_dir)

When the fine-tuning completes, we can generate molecules with high penalized logP scores with the trained model. The generated molecules are represented by a list of rdkit.Chem.Mol objects.

In [ ]:
from dig.ggraph.evaluation import PropOptEvaluator
checkpoint_path = 'prop_opt_plogp/prop_opt_net_199.pth'
n_mols = 100
mols = runner.run_prop_opt(model_conf_dict=conf['model'], checkpoint_path=checkpoint_path,
    n_mols=n_mols, num_min_node=conf['num_min_node'], num_max_node=conf['num_max_node'],
    temperature=conf['temperature'], atomic_num_list=conf['atom_list'])


Finally, we can call the evaluator to find the molecules with top-3 penalized logP scores among all generated molecules.

In [ ]:
from dig.ggraph.evaluation import PropOptEvaluator
evaluator = PropOptEvaluator()
input_dict = {'mols': mols}
print('Evaluating...')
evaluator.eval(input_dict)